In [115]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

In [2]:
mongo_client=MongoClient()
db=mongo_client['OPTIONCHAIN']

In [191]:
###############This take time to get data from DB ###########
###############Run it again if you want to fetch fresh data from db######
# dforignal=pd.DataFrame(list(db.NIFTY.find()))
# dforignal.drop("_id",axis=1,inplace=True)

In [192]:
dforignal.head()

,CallDelta,CallIV,CallLTP,CallVega,PutDelta,PutIV,PutLTP,PutVega,Strikes,datetime,expiry_date,fut_price,iv,spot
0,0.00,0.00,0.00,0.00,-1.14,31.25,3.95,93.62,8900.0,2019-01-01 09:20:00,31JAN2019,10878.9,16.33,10835.85
1,0.00,0.00,1860.00,0.00,-1.21,29.77,4.00,98.29,9000.0,2019-01-01 09:20:00,31JAN2019,10878.9,16.33,10835.85
2,0.00,0.00,1465.80,0.00,-1.94,25.00,5.70,147.92,9400.0,2019-01-01 09:20:00,31JAN2019,10878.9,16.33,10835.85
3,0.00,0.00,1370.95,0.00,-2.37,24.17,6.90,175.13,9500.0,2019-01-01 09:20:00,31JAN2019,10878.9,16.33,10835.85
4,92.68,30.85,1311.60,435.33,-2.67,22.87,7.45,193.23,9600.0,2019-01-01 09:20:00,31JAN2019,10878.9,16.33,10835.85


In [193]:
dfc=dforignal.copy()


In [46]:
dfc['datetime']=pd.to_datetime(dfc['datetime'])
dfc['expiry_date']=pd.to_datetime(dfc['expiry_date'])

In [49]:
dfc.head()

,CallDelta,CallIV,CallLTP,CallVega,PutDelta,PutIV,PutLTP,PutVega,Strikes,datetime,expiry_date,fut_price,iv,spot
0,0.00,0.00,0.00,0.00,-1.14,31.25,3.95,93.62,8900.0,2019-01-01 09:20:00,2019-01-31,10878.9,16.33,10835.85
1,0.00,0.00,1860.00,0.00,-1.21,29.77,4.00,98.29,9000.0,2019-01-01 09:20:00,2019-01-31,10878.9,16.33,10835.85
2,0.00,0.00,1465.80,0.00,-1.94,25.00,5.70,147.92,9400.0,2019-01-01 09:20:00,2019-01-31,10878.9,16.33,10835.85
3,0.00,0.00,1370.95,0.00,-2.37,24.17,6.90,175.13,9500.0,2019-01-01 09:20:00,2019-01-31,10878.9,16.33,10835.85
4,92.68,30.85,1311.60,435.33,-2.67,22.87,7.45,193.23,9600.0,2019-01-01 09:20:00,2019-01-31,10878.9,16.33,10835.85


In [66]:
dfc.set_index('datetime',inplace=True)

In [80]:
# ### Get the Data for whole Expiry
# expiry_df=dfc.loc[(dfc['expiry_date']=='2019-01-31')]

In [70]:
# ### To base of 50 
# def custom_round(x, base=5):
#     return base * round(float(x)/base)

# df = pd.Series([10835.85]).apply(lambda x: custom_round(x, base=50.0))
# custom_round(10835.85, base=50.00)

In [149]:
def get_expiry_data(expiry):
    expiry_df=dfc.loc[(dfc['expiry_date']==expiry)]
    atm_temp=expiry_df.loc[((expiry_df.index==expiry+' 09:20:00'))]['fut_price'].unique()[0]
    expiry_atm=50.00 * round(float(atm_temp)/50.00)
    return expiry_df,expiry_atm




In [169]:
#atm call and put for whole expiry series
def get_ATMstrike_data(data_after_date,expiry_date):
    ###Get whole expiry data and fut price at 9:20 on day of expiry.
    expiry_df,atm=get_expiry_data(expiry_date)
    atm_call=expiry_df.loc[(expiry_df['Strikes']==atm)][['CallLTP']]
    atm_put=expiry_df.loc[(expiry_df['Strikes']==atm)][['PutLTP']]
    atm_put['20maput']=atm_put['PutLTP'].rolling(20).mean()
    atm_put['50maput']=atm_put['PutLTP'].rolling(50).mean()
    atm_put['100maput']=atm_put['PutLTP'].rolling(100).mean()
    atm_put.replace(to_replace=np.nan,value=0)
    atm_call['20macall']=atm_call['CallLTP'].rolling(20).mean()
    atm_call['50macall']=atm_call['CallLTP'].rolling(50).mean()
    atm_call['100macall']=atm_call['CallLTP'].rolling(100).mean()
    atm_call.replace(to_replace=np.nan,value=0)
    atm_options=atm_call.merge(atm_put,on='datetime')
    atm_options.replace(to_replace=np.nan,value=0,inplace=True)
    if data_after_date:
        atm_options=atm_options.loc[atm_options.index>data_after_date]
    else:
        atm_options=atm_options.loc[atm_options.index>expiry_date]
    return atm_options

In [147]:
# ## To get fut_price ATM at 9:20 on day of expiry 
# def custom_round(x, base=5):
#     return base * round(float(x)/base)
# atm_temp=expiry_df.loc[((expiry_df.index=='2019-01-31 09:20:00'))]['fut_price'].unique()[0]
# atm=custom_round(atm_temp, base=50.00)

In [171]:
#itm call and put for whole expiry series
def get_ITMstrike_data(data_after_date,expiry_date):
    ###Get whole expiry data and fut price at 9:20 on day of expiry.
    expiry_df,itm=get_expiry_data(expiry_date)
    itm_call_strike=itm-50.00
    itm_put_strike=itm+50.00
    print(itm,itm_call_strike,itm_put_strike)
    itm_call=expiry_df.loc[(expiry_df['Strikes']==itm_call_strike)][['CallLTP']]
    itm_put=expiry_df.loc[(expiry_df['Strikes']==itm_put_strike)][['PutLTP']]
    itm_put['20maput']=itm_put['PutLTP'].rolling(20).mean()
    itm_put['50maput']=itm_put['PutLTP'].rolling(50).mean()
    itm_put['100maput']=itm_put['PutLTP'].rolling(100).mean()
    itm_put.replace(to_replace=np.nan,value=0)
    itm_call['20macall']=itm_call['CallLTP'].rolling(20).mean()
    itm_call['50macall']=itm_call['CallLTP'].rolling(50).mean()
    itm_call['100macall']=itm_call['CallLTP'].rolling(100).mean()
    itm_call.replace(to_replace=np.nan,value=0)
    itm_options=itm_call.merge(itm_put,on='datetime')
    itm_options.replace(to_replace=np.nan,value=0,inplace=True)
    if data_after_date:
        itm_options=itm_options.loc[itm_options.index>data_after_date]
    else:
        itm_options=itm_options.loc[itm_options.index>expiry_date]
    return itm_options

In [184]:
### This whole section to plot charts in Dash
def genrate_graphs():
    app = JupyterDash(__name__)
    app.layout = html.Div(children=[
            html.H1(children='Hello Dash'),

            html.Div(children='''
                Dash: A web application framework for your data.
            '''),
            dcc.Checklist(
                id='toggle-rangeslider',
                options=[{'label': 'Include Rangeslider', 
                          'value': 'slider'}],
                value=['slider']
            ),
            dcc.Graph(
                id='graph',
                # figure=fig
            )
        ])
    @app.callback(Output("graph", "figure"), [Input("toggle-rangeslider", "value")])
    def show_chart(value):
        fig=make_subplots(rows=2,cols=1,column_widths=[1.0],
                                  row_heights=[1.0,1.0],shared_xaxes=True,vertical_spacing=0.01,
                                  specs=[
                                      [{"type":"xy"}],#None]])
                                      [{"type":"xy"}],#None]]
                                        ]
                                 )
 
        fig.add_trace(go.Scatter(x=atm_options.index,y=atm_options['CallLTP'], name="ATMCall",mode='lines'),row=1,col=1)
        fig.add_trace(go.Scatter(x=atm_options.index,y=atm_options['50macall'], name="ATMCall50ma",mode='lines'),row=1,col=1)
        fig.add_trace(go.Scatter(x=atm_options.index,y=atm_options['PutLTP'], name="ATMPut",mode='lines'),row=1,col=1)
        fig.add_trace(go.Scatter(x=atm_options.index,y=atm_options['50maput'], name="ATMPut50ma",mode='lines'),row=1,col=1)
        
        fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['CallLTP'], name="ITMCall",mode='lines'),row=2,col=1)
        fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['50macall'], name="ITMCall50ma",mode='lines'),row=2,col=1)
        fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['PutLTP'], name="ITMPut",mode='lines'),row=2,col=1)
        fig.add_trace(go.Scatter(x=itm_options.index,y=itm_options['50maput'], name="ITMPut50ma",mode='lines'),row=2,col=1)
        # fig.add_trace(go.Scatter(x=atm_options.index,y=atm_options['PutLTP'], name="Put",mode='lines'),row=2,col=1)
        fig.update_xaxes(row=1,col=1,
                         # rangeselector=dict(buttons=list(
                         #     [dict(count=1, label="1m", step="month", stepmode="backward"),
                         #      dict(count=6, label="6m", step="month", stepmode="backward"),
                         #      dict(count=1, label="YTD", step="year", stepmode="todate"),
                         #      dict(count=1, label="1y", step="year", stepmode="backward"),
                         #      dict(step="all")
                         #     ])
                         #                   ),
                         rangebreaks=[
                             dict(bounds=["sat", "mon"]),#hide weekends
                             dict(bounds=[15.5, 9.25], pattern="hour")
                         ]
                        )
                # rangebreaks=[
            #     dict(bounds=[15, 9], pattern="hour"), #hide hours outside of 9am-5pm
            # ]
        fig.update_layout(autosize=False,width=1200,height=800,
                          margin=dict(l=50,r=50,b=100,t=100,pad=4),
                          paper_bgcolor="LightSteelBlue",)
        return fig
    app.run_server(mode='jupyterlab',port = 8097)

In [185]:
expiry_date='2019-01-31'
after='2019-01-31'
atm_options=get_ATMstrike_data(after,expiry_date)
atm_options.replace(to_replace=0,method='ffill',inplace=True)
itm_options=get_ITMstrike_data(after,expiry_date)
itm_options.replace(to_replace=0,method='ffill',inplace=True)

genrate_graphs()

10700.0 10650.0 10750.0
